In [1]:
import random
import string
import RNA
import numpy as np
import os
import subprocess
from PIL import Image 
from IPython.display import SVG, display
from collections import deque 
import difflib
import sys

import logging # doesn't print anything, unless a root logger is created
logger = logging.getLogger()
# logger.setLevel(logging.DEBUG)
err = lambda x: logging.error(x)
deb = lambda x: logging.debug(x)
inf = lambda x: logging.info(x)
inf(f'hello!')

sys.path.append(os.path.dirname("/home/mescalin/maxf/pathfinder_project/src"))
import importlib # always reload all .py files / observe changes
import p_tables_helper
import pathfinder
import helper

importlib.reload(p_tables_helper)
importlib.reload(pathfinder)
importlib.reload(helper)

# i_list = list(set([x[0] for x in input]))
# j_list = list(set([x[1] for x in input]))
# for i in i_list:
#     print (i, j_list[-1])
# for i in j_list:
#     print (i, i_list[-1])


<module 'helper' from '/mnt/c/Users/Max/Google Drive/pathfinder/src/helper.py'>

In [3]:
def moves_to_en(moves):
    new_ptable = current_ptable.copy()
    for i,j in moves:
        new_ptable[i] = j
        new_ptable[j] = i
    return fc.eval_structure_pt(new_ptable)/100

In [ ]:
input = [(1, 63), (1, 65), (1, 67), (1, 69), (7, 11), (7, 12), (7, 14), (7, 48), (7, 50), (10, 13), (10, 51), (10, 52), (10, 53), (10, 54), (11, 49), (11, 51), (11, 52), (11, 53), (11, 54), (11, 56), (12, 49), (12, 51), (12, 52), (12, 53), (12, 54), (12, 56), (13, 48), (13, 50), (14, 49), (14, 51), (14, 52), (14, 53), (14, 54), (14, 56), (19, 42), (20, 42), (21, 42), (25, 34), (26, 29), (26, 30), (26, 33), (26, 34), (26, 35), (26, 37), (27, 31), (27, 32), (27, 36), (28, 34), (29, 32), (29, 36), (30, 36), (31, 34), (31, 35), (31, 37), (32, 35), (32, 37), (33, 36), (48, 51), (48, 52), (48, 53), (48, 54), (48, 56), (50, 53), (50, 54), (50, 56), (63, 68), (64, 67), (64, 69), (65, 68)]
sequence = "UCACUGAGGCUUGUUCGCAAAUCACUGCAAUUAGAUAUGACUCACGAUAUGGGGCACGGUGCAUACAUAC"
current_ptable = [70, 0, 61, 60, 59, 58, 57, 0, 0, 0, 0, 0, 0, 0, 0, 47, 46, 45, 0, 0, 0, 0, 40, 39, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 23, 22, 0, 0, 0, 0, 17, 16, 15, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 5, 4, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
fc = RNA.fold_compound(sequence)
current_en = fc.eval_structure_pt(current_ptable)/100


# input = [(1, 4), (1, 7), (1, 8), (1, 38), (1, 41), (1, 43), (2, 6), (2, 23), (2, 36), (2, 37), (2, 39), (2, 40), (2, 42), (2, 45), (2, 46), (3, 7), (3, 8), (3, 38), (3, 41), (3, 43), (4, 23), (4, 36), (4, 37), (4, 39), (4, 40), (4, 42), (4, 45), (4, 46), (5, 23), (5, 39), (5, 45), (6, 38), (6, 41), (6, 43), (7, 23), (7, 36), (7, 37), (7, 39), (7, 40), (7, 42), (7, 45), (7, 46), (8, 23), (8, 36), (8, 37), (8, 39), (8, 40), (8, 42), (8, 45), (8, 46), (21, 39), (21, 45), (22, 39), (22, 45), (23, 38), (23, 41), (23, 43), (23, 44), (36, 41), (36, 43), (37, 41), (37, 43), (38, 42), (38, 45), (38, 46), (39, 43), (39, 44), (40, 43), (41, 45), (41, 46), (43, 46)]
# sequence = "CGCGACGGCUACGCGACGGCAAUGCCGUUGCGAAGCCGUCGCGAUC"
# current_ptable = [46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
# fc = RNA.fold_compound(sequence)
# current_en = fc.eval_structure_pt(current_ptable)/100

print (current_en)

start_values = [x[0] for x in input]
end_values   = [x[0] for x in input]

# input is a list of tuples which can potentially added [(1, 63), (1, 65), (1, 67), (1, 69), (7, 11), (7, 12)]
# lets find out which tuple combinations have potential to lower the energy

combinations = [[moves_to_en([(x[0],x[1])]),[(x[0],x[1])]] for x in input]
all_candidates = []
while True:    
    all_candidates += combinations
    next_combinations = []    
    for current_energy, candidate_list in combinations:
        for next_candidate in input:            
            next_candidate_list = candidate_list.copy()            
           
            if next_candidate in candidate_list:
                continue                
            start = candidate_list[-1][0]
            end = candidate_list[-1][1]            
            # next candidate has to fit into the last one
            if next_candidate[0]<=start or next_candidate[1]>=end:
                continue         
            next_candidate_list += [next_candidate]    
            next_combinations.append([moves_to_en(next_candidate_list), next_candidate_list])
    
    if next_combinations == []:
        break
    next_combinations.sort()
    combinations = next_combinations[0:20]

all_candidates.sort()

for candidate in all_candidates:
    print (candidate)
# print (all_candidates)







# for i,j in input:
#     new_ptable = current_ptable.copy()
#     new_ptable[i] = j
#     new_ptable[j] = i
#     en = fc.eval_structure_pt(new_ptable)/100
#     print (i,j,en)


In [72]:
sequence   = "UCUUGAACCCAUGGCGUUUUCAACCGACAUCCUGCUCCCGCAAUCACCUAGGUUAAGGGUUCUUCAAGGA"
structure  = "((((((((((.((((.................(((....))).........)))).)))))...)))))."

sequence = "CGCGACGGCUACGCGACGGCAAUGCCGUUGCGAAGCCGUCGCGAUC"
structure = ".............................................."
# 
# #dsra
# sequence   = "ACACAUCAGAUUUCCUGGUGUAACGAAUUUUUUAAGUGCUUCUUGCUUAAGCAAGUUUCAUCCCGACCCCCUCAGGGUCGGGAUU"
# structure  = "..(((((((.....)))))))......................................((((((((((.....))))))))))."
# structure  = "..(((((((.....)))))))...(((((((((((...........))))).)))))).((((((((((.....))))))))))."
# structure  = "..(((((((.....)))))))xxxxxxxxxxxx(.............)xxxxxxxxxxx((((((((((.....))))))))))."



#     cmd = f'printf "{sequence}" | RNAsubopt -p 2'
# cmd = f'printf "{sequence}\n{structure}\n" | RNAsubopt -C −−enforceConstraint'
# cmd = f'printf "{sequence}\n{structure}\n" | RNAsubopt -C -p 1 −−enforceConstraint'

cmd = f'printf "{sequence}\n{structure}\n" | RNAsubopt -C −−enforceConstraint -e 2'
result = subprocess.check_output(cmd, shell=True, encoding="utf8")
subopt_list = result.split("\n")
  
subopt_structure = subopt_list[1].split()[0]

fc = RNA.fold_compound(sequence)

current_ptable = RNA.ptable_from_string(structure)
subopt_ptable = RNA.ptable_from_string(subopt_structure)

candidates = []
for i in range(1, len(current_ptable)):
    if current_ptable[i] == subopt_ptable[i]: continue
    if subopt_ptable[i] == 0: continue
    if i>subopt_ptable[i]: continue # closing bracket
    candidates.append((subopt_ptable[i], i))    
print (candidates)


..............................................
~~~
((((((((((.(((((((((...))))))))).))))))))))...
(46, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 0, 32, 31, 30, 29, 28, 27, 26, 25, 24, 0, 0, 0, 20, 19, 18, 17, 16, 15, 14, 13, 12, 0, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 0, 0)
(46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
[(43, 1), (42, 2), (41, 3), (40, 4), (39, 5), (38, 6), (37, 7), (36, 8), (35, 9), (34, 10), (32, 12), (31, 13), (30, 14), (29, 15), (28, 16), (27, 17), (26, 18), (25, 19), (24, 20)]


In [4]:
import RNA
 
sequence = "GGGGAAAACCCC"
 
# Set global switch for unique ML decomposition
RNA.cvar.uniq_ML = 1
 
subopt_data = { 'counter' : 1, 'sequence' : sequence }
 
# Print a subopt result as FASTA record
def print_subopt_result(structure, energy, data):
    if not structure == None:
        print (">subopt %d" % data['counter'])
        print ("%s" % data['sequence'])
        print ("%s [%6.2f]" % (structure, energy))
        # increase structure counter
        data['counter'] = data['counter'] + 1
 
# Create a 'fold_compound' for our sequence
a = RNA.fold_compound(sequence)
 
# Enumerate all structures 500 dacal/mol = 5 kcal/mol arround
# the MFE and print each structure using the function above
a.subopt_cb(500, print_subopt_result, subopt_data);

>subopt 1
GGGGAAAACCCC
(((......))) [ -2.70]
>subopt 2
GGGGAAAACCCC
((((....)))) [ -5.40]
>subopt 3
GGGGAAAACCCC
.(((.....))) [ -2.40]
>subopt 4
GGGGAAAACCCC
(((.....))). [ -4.20]
>subopt 5
GGGGAAAACCCC
.((......)). [ -0.40]
>subopt 6
GGGGAAAACCCC
.(((....))). [ -3.10]
>subopt 7
GGGGAAAACCCC
((......)).. [ -1.20]
>subopt 8
GGGGAAAACCCC
.((.....)).. [ -1.10]


In [28]:
import RNA
 
sequence = "UGGGAAUAGUCUCUUCCGAGUCUCGCGGGCGACGGGCGAUCUUCGAAAGUGGAAUCCGUACUUAUACCGCCUGUGCGGACUACUAUCCUGACCACAUAGU"
 
"""
A simple callback function that stores
a structure sample into a list
"""
def store_structure(s, data):
    if s:
        data.append(s)
 
 
"""
First we prepare a fold_compound object
"""
 
# create model details
md = RNA.md()
 
# activate unique multibranch loop decomposition
md.uniq_ML = 1
 
# create fold compound object
fc = RNA.fold_compound(sequence, md)
 
# compute MFE
(ss, mfe) = fc.mfe()
 
# rescale Boltzmann factors according to MFE
fc.exp_params_rescale(mfe)
 
# compute partition function to fill DP matrices
fc.pf()
 
"""
Now we are ready to perform Boltzmann sampling
"""
 
# 1. backtrace a single sub-structure of length 10
print ("%s" % fc.pbacktrack5(10))
 
 
# 2. backtrace a single sub-structure of length 50
print ("%s" % fc.pbacktrack5(50))
 
# 3. backtrace multiple sub-structures of length 10 at once
for s in fc.pbacktrack5(20, 10):
    print ("%s" % s)
 
# 4. backtrace multiple sub-structures of length 50 at once
for s in fc.pbacktrack5(100, 50):
    print ("%s" % s)
 
# 5. backtrace a single structure (full length)
print ("%s" % fc.pbacktrack())
 
# 6. backtrace multiple structures at once
for s in fc.pbacktrack(100):
    print ("%s" % s)
 
# 7. backtrace multiple structures non-redundantly
for s in fc.pbacktrack(100, RNA.PBACKTRACK_NON_REDUNDANT):
    print ("%s" % s)
 
# 8. backtrace multiple structures non-redundantly (with resume option)
num_samples = 500
iterations  = 15
d           = None # pbacktrack memory object
s_list      = list()
 
for i in range(0, iterations):
    d, ss   = fc.pbacktrack(num_samples, d, RNA.PBACKTRACK_NON_REDUNDANT)
    s_list  = s_list + list(ss)
 
for s in s_list:
    print ("%s" % s)
 
# 9. backtrace multiple sub-structures of length 50 in callback mode
ss  = list()
i   = fc.pbacktrack5(100, 50, store_structure, ss)
 
for s in ss:
    print ("%s" % s)
 
# 10. backtrace multiple full-length structures in callback mode
ss  = list()
i   = fc.pbacktrack(100, store_structure, ss)
 
for s in ss:
    print ("%s" % s)
 
# 11. non-redundantly backtrace multiple full-length structures in callback mode
ss  = list()
i   = fc.pbacktrack(100, store_structure, ss, RNA.PBACKTRACK_NON_REDUNDANT)
 
for s in ss:
    print ("%s" % s)
 
# 12. non-redundantly backtrace multiple full length structures
# in callback mode with resume option
ss = list()
d  = None # pbacktrack memory object
 
for i in range(0, iterations):
    d, i = fc.pbacktrack(num_samples, store_structure, ss, d, RNA.PBACKTRACK_NON_REDUNDANT)
 
for s in ss:
    print ("%s" % s)

..........
.((((....))))(((.(((((.(.((.....)).).)).))).)))...
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
..............(.((((..((((.........)))).)).)).)...
..(((....)))((..((((((.(.((.....)).).)))..)))..)).
.(((........))).((((..((((.(....)..))))..)))).....
.(((.....)))....((((..((((.(....)..))))..)))).....
............(((.((((((.(.((.....)).).))..)))).))).
.((((....))))(((..((..((((.........)))).))..)))...
.((((....))))((.((((((.(.((.....)).).)).).)))))...
...((...(((((.((((.......)))).))..)))..))..(....).
..((((..(((((.((((.......)))).))..)))..).)))......
.((((....))))(((.(((..((((.........)))).))).)))...
..(((...(((...((((.((((...))))..)))).))).)))......
.((((....))))...((((..((((.(....)..))))..)))).....
..(((.........)))(((..((((.(....)..)))).))).......
.((((....)))).........((((.........))))...........
.(((

1